In [11]:
import pandas as pd
import tensorflow as tf
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]
column_labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5", "Ball Delta", "Ball Y Delta", "LS Delta",
    "Ball 1 Repeat Last 5", "Ball 2 Repeat Last 5", "Ball 3 Repeat Last 5", "Ball 4 Repeat Last 5", "Ball 5 Repeat Last 5", "Lucky Star 1 Repeat Last 5", "Lucky Star 2 Repeat Last 5",
    "Ball 1 Repeat Last 10", "Ball 2 Repeat Last 10", "Ball 3 Repeat Last 10", "Ball 4 Repeat Last 10", "Ball 5 Repeat Last 10", "Lucky Star 1 Repeat Last 10", "Lucky Star 2 Repeat Last 10",
    "Ball 1 Repeat Last 20", "Ball 2 Repeat Last 20", "Ball 3 Repeat Last 20", "Ball 4 Repeat Last 20", "Ball 5 Repeat Last 20", "Lucky Star 1 Repeat Last 20", "Lucky Star 2 Repeat Last 20"
]
column_indices = {name: i for i, name in enumerate(column_labels)}


df = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions", header=0)

df[column_labels].to_csv("euromillions-dataset.csv", index=None)

Num GPUs Available:  0


In [12]:
df = pd.read_csv('euromillions-dataset.csv')
df["Ball Y Delta 2"] = df["Ball Y Delta"]
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Ball Delta,Ball Y Delta,LS Delta,...,Lucky Star 1 Repeat Last 10,Lucky Star 2 Repeat Last 10,Ball 1 Repeat Last 20,Ball 2 Repeat Last 20,Ball 3 Repeat Last 20,Ball 4 Repeat Last 20,Ball 5 Repeat Last 20,Lucky Star 1 Repeat Last 20,Lucky Star 2 Repeat Last 20,Ball Y Delta 2
0,5,9,18,20,29,41,48,4.168333,2.727636,4.0,...,1.0,3.0,1.0,2.0,5.0,1.0,1.0,2.0,6.0,2.727636
1,6,7,12,27,36,37,42,4.555217,2.383275,1.0,...,2.0,1.0,3.0,1.0,0.0,2.0,2.0,5.0,3.0,2.383275
2,4,12,20,33,35,41,47,3.913119,5.126402,8.0,...,2.0,2.0,1.0,5.0,2.0,0.0,3.0,2.0,5.0,5.126402
3,1,9,3,19,29,35,37,4.974937,3.768289,8.0,...,0.0,3.0,1.0,3.0,4.0,1.0,1.0,2.0,6.0,3.768289
4,8,12,19,24,26,28,33,1.903943,2.986637,4.0,...,2.0,2.0,2.0,2.0,0.0,1.0,4.0,4.0,4.0,2.986637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,6,9,1,19,36,38,49,6.791539,2.814249,3.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.814249
301,9,10,10,25,29,34,45,4.918079,3.143247,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143247
302,2,4,6,16,18,39,47,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,7,12,5,14,35,36,39,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# --- 1. Data Preparation ---

def create_sequences(data, seq_length, pred_cols):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)].values
        y = data.iloc[i + seq_length, :pred_cols].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# --- 2. Load and Preprocess Data ---

# Example DataFrame creation
num_rows = 2000
num_cols = 32
random_seed = np.random.rand(num_rows, num_cols)

# Parameters
seq_length = 7  # Number of past rows to look at
pred_cols = 7  # Number of columns to predict
all_cols = 32
random_seed_length = 7 # Length of random seed

# Scale the data
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df[::-1]), columns=df.columns)

# Create sequences
X, y = create_sequences(df, seq_length, pred_cols)

# Split into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# Input for the sequence
sequence_input = tf.keras.layers.Input(shape=(seq_length, all_cols))

# Input for the random seed
random_seed_input = tf.keras.layers.Input(shape=(random_seed_length,))

# Reshape the random seed to make it compatible for concatenation or addition
random_seed_reshaped = tf.keras.layers.Reshape((random_seed_length, 1))(random_seed_input)


In [13]:
scaler.inverse_transform(X[-1])

array([[ 6.00000000e+00,  9.00000000e+00,  1.00000000e+01,
         1.40000000e+01,  2.10000000e+01,  3.30000000e+01,
         5.00000000e+01,  5.57897840e+00,  6.56048779e+00,
         3.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         2.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  3.00000000e+00,  3.00000000e+00,
         2.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         2.00000000e+00,  4.00000000e+00,  4.00000000e+00,
         6.00000000e+00,  6.56048779e+00],
       [ 1.10000000e+01,  1.20000000e+01,  6.00000000e+00,
         1.00000000e+01,  1.60000000e+01,  2.30000000e+01,
         2.40000000e+01,  2.52487623e+00,  5.77234788e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.11022302e-16,
         2.00

In [63]:
# for testing sequences
# df = df[::-1]
# print(y[-1])
# print(df.iloc[-1].values[:7])

[ 6.  7. 12. 27. 36. 37. 42.]
[ 6.  7. 12. 27. 36. 37. 42.]


In [14]:
# --- 3. Model Definition ---

# First 1D Convolutional layer
conv1_1 = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(sequence_input)
conv1_2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1_1)

# LSTM layer
lstm1_1 = tf.keras.layers.LSTM(64, return_sequences=True)(conv1_2)
lstm1_2 = tf.keras.layers.Dropout(0.05)(lstm1_1)

# Reshape for Dense output
resh1_1 = tf.keras.layers.Reshape((64, 2))(lstm1_2)

# Second 1D Convolutional layer
conv2_1 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(resh1_1)
conv2_2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2_1)

# Flatten for Dense output
flat1_1 = tf.keras.layers.Flatten()(conv2_2)

dense = tf.keras.layers.Dense(pred_cols*2)(flat1_1)

concat = tf.keras.layers.Concatenate()([dense, random_seed_input])

output = tf.keras.layers.Dense(pred_cols)(concat)

# Create the model with two inputs
lstm_model = tf.keras.models.Model(inputs=[sequence_input, random_seed_input], outputs=output)

In [15]:

# --- 4. Loss Function with Emphasis ---

def custom_loss(y_true, y_pred, random_target):
    # Calculate loss
    squared_error = tf.square(y_true - y_pred)
    weighted_mse_loss = tf.reduce_mean(squared_error)

    # MSE between prediction and random target
    mse_loss_random = tf.reduce_mean(tf.square(y_pred - random_target))

    # Combine the two losses (you can adjust the weights)
    return 0.3 * weighted_mse_loss + 0.7 * mse_loss_random

def generate_random_target(batch_size):
    targets = []
    for _ in range(batch_size):
        first_two = np.random.randint(1, 13, size=2) / 12
        last_five = np.random.randint(1, 51, size=5) / 51
        target = np.concatenate([first_two, last_five])
        targets.append(target)
    return np.array(targets)

# --- 5. Compile the Model ---

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=custom_loss)  # Use the custom weighted loss




In [16]:
X_train.shape

(268, 7, 32)

In [18]:
# --- 6. Train the Model ---

# Generate random seeds for training and validation data
train_random_seeds = np.random.randint(1, 51, size=(X_train.shape[0], random_seed_length))
val_random_seeds = np.random.randint(1, 51, size=(X_val.shape[0], random_seed_length))

# Custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

epochs = 100
batch_size = 16

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        # Get batch data

        X_batch = X_train[i:(i + batch_size)]
        y_batch = y_train[i:(i + batch_size)]
        random_seeds_batch = generate_random_target(X_batch.shape[0])

        with tf.GradientTape() as tape:
            # Get predictions
            predictions = lstm_model([X_batch, random_seeds_batch], training=True)

            # Combine the two losses (you can adjust the weights)
            loss = custom_loss(y_batch, predictions, random_seeds_batch)

        # Compute and apply gradients
        gradients = tape.gradient(loss, lstm_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, lstm_model.trainable_variables))

        epoch_loss += loss

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (X_train.shape[0] // batch_size)}")



Epoch 1/100, Loss: 0.42115163803100586
Epoch 2/100, Loss: 0.41366010904312134
Epoch 3/100, Loss: 0.4129461646080017
Epoch 4/100, Loss: 0.41336655616760254
Epoch 5/100, Loss: 0.41714784502983093
Epoch 6/100, Loss: 0.4160464107990265
Epoch 7/100, Loss: 0.4104626774787903
Epoch 8/100, Loss: 0.4132768511772156
Epoch 9/100, Loss: 0.41803470253944397
Epoch 10/100, Loss: 0.41080591082572937
Epoch 11/100, Loss: 0.40968355536460876
Epoch 12/100, Loss: 0.4125472605228424
Epoch 13/100, Loss: 0.4070233106613159
Epoch 14/100, Loss: 0.4078224003314972
Epoch 15/100, Loss: 0.40078631043434143
Epoch 16/100, Loss: 0.4148121178150177
Epoch 17/100, Loss: 0.40732043981552124
Epoch 18/100, Loss: 0.40573781728744507
Epoch 19/100, Loss: 0.4041373133659363
Epoch 20/100, Loss: 0.3978375196456909
Epoch 21/100, Loss: 0.4039655029773712
Epoch 22/100, Loss: 0.4075356721878052
Epoch 23/100, Loss: 0.4046784043312073
Epoch 24/100, Loss: 0.40537047386169434
Epoch 25/100, Loss: 0.404887318611145
Epoch 26/100, Loss: 0.40

In [19]:
# --- 8. Evaluate the Model ---

val_loss = 0
for i in range(0, X_val.shape[0], batch_size):
    X_batch = X_val[i:i + batch_size]
    y_batch = y_val[i:i + batch_size]
    random_target_batch = generate_random_target(X_batch.shape[0])

    predictions = lstm_model([X_batch, random_target_batch], training=False)
    loss = custom_loss(y_batch, predictions, random_target_batch)
    val_loss += loss

print(f"Validation Loss: {val_loss / (X_val.shape[0] // batch_size)}")

Validation Loss: 0.7438267469406128


In [26]:
# --- 9. Make Predictions ---

# Generate a random seed for prediction
prediction_random_seed = generate_random_target(1)

# Get the last sequence from X_val and reshape it for prediction
last_sequence = X[-1].reshape(1, seq_length, all_cols)

# Make a prediction using the last sequence and the random seed
prediction = lstm_model.predict([last_sequence, prediction_random_seed])

# Inverse transform to get predictions in the original scale
prediction = scaler.inverse_transform(np.concatenate((prediction, np.zeros((prediction.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

prediction

1/1 [==============================] - 0s 13ms/step


array([[ 6.15954063,  8.87274044, 12.18904503, 21.23690918, 29.13599081,
        39.27431421, 46.03618629]])